<a href="https://colab.research.google.com/github/mhuckvale/pals0039/blob/master/Answers_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![PALS0039 Logo](https://www.phon.ucl.ac.uk/courses/pals0039/images/pals0039logo.png)](https://www.phon.ucl.ac.uk/courses/pals0039/)

#Exercise 3.3 Answers

In this exercise we train a network to classify vowels from formant data.

(a) Load in vowels data set and prepare for learning. Run the code then add comments to explain how it works.

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

# use pandas to read the data set
df=pd.read_csv("https://www.phon.ucl.ac.uk/courses/pals0039/data/vowels.csv")

# display the start of the data set
df.head()

---
(b) Convert SEX column to a number. Run the code then add comments to explain how it works.

In [ ]:
# convert SEX column from string to categories
df["SEX"]=df.SEX.astype('category')
# convert categories to numerical codes
df["SEX"]=df.SEX.cat.codes

---
(c) Copy all numbers into numpy array and standardise. Run the code then add comments to explain how it works.

In [ ]:
# select columns F1, F2, SEX and HEIGHT for all rows
X=np.array(df.iloc[:,3:7])
print(X)

# standardise by subtracting mean and dividing by standard deviation
for i in range(X.shape[1]):
  X[:,i]=(X[:,i]-np.mean(X[:,i]))/np.std(X[:,i])
print(X)


---
(d) Convert vowels to one hot coding. Run the code then add comments to explain how it works.

In [ ]:
# convert vowel column to categories
df["VOWEL"]=df.VOWEL.astype('category')
# convert vowel categories to numerical codes
vowels=df.VOWEL.cat.codes
print(vowels[:10])
# convert numerical codes to one-hot coding
vowelcode = to_categorical(vowels)
print(vowelcode[:10])
# record number of vowels in data set
nvowel=vowelcode.shape[1]

---
(e) Divide the data into train and test. Run the code then add comments to explain how it works.

In [ ]:
# select 80% of data for training, 20% for testing
ntrain=int(0.8*X.shape[0])
# get a random permutation of the samples
indices = np.random.permutation(X.shape[0])
# select those samples to use for training
Xtrain,Xtest = X[indices[:ntrain],:], X[indices[ntrain:],:]
# select those sampels to use for testing
ytrain,ytest = vowelcode[indices[:ntrain],:], vowelcode[indices[ntrain:],:]


---
(f) Build a classifier network with configurable number of inputs and outputs. Run the code then add comments to explain how it works.

In [ ]:
# function to build a network
def build_model(numinput,numhidden,numoutput):
  # new model
  model=Sequential()
  # add layer with numhidden units and numinput inputs
  model.add(Dense(numhidden,activation='sigmoid',input_shape=(numinput,),name="hidden_layer"))
  # add layer with numoutput outputs - build as softmax output
  model.add(Dense(numoutput,activation='softmax',name="output_layer"))
  # build a suitable optimizer
  sgd=SGD(learning_rate=0.1, momentum=0.9)
  # compile network with cross-entropy loss
  model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=["accuracy"])
  return model

---
(g) Train the network. Run the code then add comments to explain how it works.

In [ ]:
# build the network with given parameters
model=build_model(Xtrain.shape[1],32,nvowel);
# fit the model, returning history
hist=model.fit(Xtrain,ytrain,validation_data=(Xtest,ytest),epochs=100,batch_size=16,verbose=0)
# evauate the performance of the model on the training data
print(model.evaluate(Xtrain,ytrain,verbose=0))

---
(h) Plot training curves. Run the code then add comments to explain how it works.

In [ ]:
# create a large figure
plt.figure(figsize=(10,4))
# divide figure into two parts
plt.subplot(1,2,1)
# plot the training loss and validation loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend(["loss","val_loss"])
# second plot
plt.subplot(1,2,2)
# plot the training accuracy and the validation accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["accuracy","val_accuracy"])
plt.show()

---
(i) Calculate performance on test data. Run the code then add comments to explain how it works.

In [ ]:
# predict the vowel codes
vowelcodepred=model.predict(Xtest)
print(vowelcodepred)
# get the true vowels from the test labels
voweltrue=np.argmax(ytest,axis=1)
print(voweltrue)
# get the most probably vowels in each row
vowelpred=np.argmax(vowelcodepred,axis=1)
print(vowelpred)
# get the total number of vowels in the test data
total=ytest.shape[0]
# count the number of correct predictions
correct=np.sum(voweltrue==vowelpred)
print("Correct %d/%d (%.1f%%)" % (correct,total,100.0*correct/total))

---
(j) Experiment with changing the number of hidden units, the number of training epochs and the optimizer parameters to see their effect on test set performance.